<div class="clearfix" style="padding: 10px; padding-left: 0px">

<a href="https://github.com/ProjectPyRhO/PyRhO"><img src="https://raw.githubusercontent.com/ProjectPyRhO/PyRhO/master/pyrho/gui/PyRhO_logo.png" alt="PyRhO logo" title="PyRhO: A Multiscale Optogenetics Simulation Platform. Logo by Pepe Herrero" width="175px" style="display: inline-block; margin-top: 5px;"></a>

<a href="https://jupyter.org/"><img src="https://raw.githubusercontent.com/jupyter/nature-demo/master/images/jupyter-logo.png" alt="Jupyter logo" title="Jupyter" width="150px" class="pull-right" style="display: inline-block; margin: 0px;"></a>
</div>

## Welcome to Prometheus: Modelling as a Service!

Prometheus is a web portal for modelling and simulating rhodopsins. It's a temporary way for you to try out [PyRhO](https://github.com/ProjectPyRhO/PyRhO) (computational tools for optogenetics) in an IPython/[Jupyter](https://jupyter.org/) notebook with absolutely no installation or set-up required. You can find more information about PyRhO in our [open-access paper](https://dx.doi.org/10.3389/fninf.2016.00008). 

This project was conceived of and developed by Benjamin Evans and Konstantin Nikolic in the [Bio-modelling group](https://www.imperial.ac.uk/bio-modelling) at [Imperial College London](https://www.imperial.ac.uk/). The work was kindly supported by the UK BBSRC grant: BB/L018268/1, the BBSRC Impact Acceleration Award and the UK EPSRC grant: EP/N002474/1. 

If you find this service useful or have ideas for improvements, all comments are welcome at <a href="mailto:projectpyrho@gmail.com?subject=I just tried Prometheus!">projectpyrho@gmail.com</a>. For the latest updates, follow us on twitter [@ProjectPyRhO](https://twitter.com/ProjectPyRhO)! 

### Run some example Python code or use the web-based GUI!

This Notebook Server was **launched just for you** with [tmpnb](https://github.com/jupyter/tmpnb)! It's a temporary "sandbox" environment where everything is configured and ready to go. Try launching the GUI below or running some of the code examples to see what PyRhO can offer you. 

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p>**WARNING**</p>

<p>Don't rely on this server for anything you want to last - your server will be *deleted after 10 minutes of inactivity*.</p>
<p>This is a pilot study running on a server with limited resources so if performance is slow, please try again later.</p>
</div>

If you would like to keep your work, in the `File` menu select `Download as` and then choose a format of your choice to save the notebook. If you select `Notebook (.ipynb)`, you can [install PyRhO on your computer](http://www.imperial.ac.uk/bio-modelling/pyrho/overview--installation/) and continue where you left off. 

### Quickstart -- Graphical User Interface
To run the code below:

1. Click on the cell to select it.
2. Press `SHIFT+ENTER` on your keyboard or press the play button (<button class='fa fa-play icon-play btn btn-xs btn-default'></button>) in the toolbar above.

Feel free to create new cells using the plus button (<button class='fa fa-plus icon-plus btn btn-xs btn-default'></button>), or pressing `SHIFT+ENTER` while this cell is selected.

In [ ]:
%matplotlib notebook
from pyrho import *
loadGUI()

### Advanced usage

In [ ]:
# Import the module and set figure rendering to be inline. (You will not see any output yet.)
# If you already used the GUI, this cell does not need to be run. 
%matplotlib notebook
from pyrho import *

### Model fitting process
N.B. Model-fitting is optional but fitting the six-state model takes around 12-13 minutes. 

In [ ]:
initParams = Parameters()
initParams.add_many(                
                # Name   Value   Vary    Min     Max     Expr
                ('g0',   2.5e4,  True,   0.0,    1e15,   None),
                ('gam',  0.05,   True,   0.00,   1,      None),
                ('phi_m',3.5e17, True,   1e15,   1e19,   None),
                ('k1',   10,     True,   0.0,    1000,   None),
                ('k2',   3,      True,   0.0,    1000,   None),
                ('p',    1,      True,   0.1,    5,      None),
                ('Gf0',  0.04,   True,   0.0,    1000,   None),
                ('k_f',  0.1,    True,   0.0,    1000,   None),
                ('Gb0',  0.02,   True,   0.0,    1000,   None),
                ('k_b',  0.15,   True,   0.0,    1000,   None),
                ('q',    1,      True,   0.1,    5,      None),
                ('Go1',  2,      True,   0.0,    1000,   None),
                ('Go2',  2,      True,   0.0,    1000,   None),
                ('Gd1',  0.1,    True,   0.0,    1000,   None),
                ('Gd2',  0.01,   True,   0.0,    1000,   None),
                ('Gr0',  3.3e-4, True,   0.0,    1000,   None),
                ('E',    0,      True,   -1000,  1000,   None),
                ('v0',   43,     True,   -1e15,  1e15,   None),
                ('v1',   17.1,   True,   -1e15,  1e15,   None))
saveData(initParams, 'initParams')
from pyrho.datasets import *
ChR2data = loadChR2()
fitParams = fitModels(ChR2data, nStates=6, params=initParams, postFitOpt=True, relaxFact=2)
# Parameters automatically saved as 'fitted{}sParams'.format(nStates)

### Command-based NEURON simulation

In [ ]:
RhO = models['6']()
Prot = protocols['step']()
Prot.phis = [1e16, 1e15, 1e14]
Sim = simulators['NEURON'](Prot, RhO)
Sim.run()
Sim.plot()

### Brian simulation

In [ ]:
import brian2 as br

### Define the rhodopsin model
nStates = '6'
origParams = modelFits[nStates]['ChR2'] # Load from pre-fit models
# Alternatively uncomment the line below to use fitted params from above
#origParams = loadData('fitted{}sParams'.format(nStates))
RhO = models[nStates](origParams)

### Define the stimulation protocol
Prot = protocols['step']()
Prot.phis = [1e18]
Prot.Vs = [None]
Prot.cycles = [[150, 100], [200, 75]]

### Define network parameters
from brian2.units import Mohm
from brian2.units.stdunits import ms, mV
N = 80
pConnect = 0.2
psp = 'v_post += 1.5*mV'
delay = 3*ms
netParams = {'tau_m':10*ms, 'R_m':70*Mohm, 'E_m':-70*mV, 'v_t0':-50*mV, 'sigma':10*mV, 't_ref':4*ms}

### Define neuron model
eqRhO = '''dv/dt = ((-I*R_m)+E_m-v)/tau_m + sigma*xi*tau_m**-0.5 : volt''' + RhO.brian_phi_t
eqLIF = '''dv/dt = (E_m-v)/tau_m + sigma*xi*tau_m**-0.5 : volt'''

### Create neuron groups - use Euler-Maruyama method for stochasticity
G0 = br.NeuronGroup(N, eqRhO, threshold='v>v_t0', reset='v=E_m', refractory='t_ref', 
                    namespace=netParams, name='Inputs', method='euler')
G0.v = 'rand()*(v_t0-E_m)+E_m'
G1 = br.NeuronGroup(N/2, eqLIF, threshold='v>v_t0', reset='v=E_m', refractory='t_ref', 
                    namespace=netParams, method='euler')
G1.v = 'rand()*(v_t0-E_m)+E_m'
G2 = br.NeuronGroup(N/4, eqLIF, threshold='v>v_t0', reset='v=E_m', refractory='t_ref', 
                    namespace=netParams, method='euler')
G2.v = 'rand()*(v_t0-E_m)+E_m'

### Create synapses
S1 = br.Synapses(G0, G1, on_pre=psp, delay=delay)
S1.connect(True, p=pConnect)
S2 = br.Synapses(G1, G2, on_pre=psp, delay=delay)
S2.connect(True, p=pConnect)

### Create monitors
monitors = {'states' : br.StateMonitor(G0, RhO.brianStateVars, record=0),   # Record states
            'I'      : br.StateMonitor(G0, 'I', record=0),                  # Record current
            'V'      : br.StateMonitor(G0, 'v', record=0),                  # Record voltage
            'spikes' : [br.SpikeMonitor(G0, name='Retina'),                 # Record spikes
                        br.SpikeMonitor(G1, name='LGN'), 
                        br.SpikeMonitor(G2, name='V1')]}

### Build the network
net = br.Network(br.collect())
net.add(monitors)

### Run the simulation
Sim = simulators['Brian'](Prot, RhO, simParams['Brian'], net, netParams, monitors)
Sim.run()
Sim.plot()

### Custom stimulus protocol

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as spline
RhO = models['6']()
Prot = protocols['custom']()
Prot.phis = [1e15, 1e16]
Sim = simulators['Python'](Prot, RhO)

def pulseGenerator(run, phi, pulse):
    """Custom interpolation function for a step pulse with damped sinusoidal oscillations"""
    pStart, pEnd = pulse
    t = np.linspace(0, pEnd-pStart, 1001, endpoint=True)
    x = phi + 0.5*phi*np.sin(0.2*np.pi*t)*np.exp(-.05*t)
    return spline(pStart + t, x, ext=1, k=5)

Prot.phi_ft = pulseGenerator

Sim.run()
Sim.plot()